In [1]:
import pandas as pd
import datetime as dt
import hvplot.pandas

In [26]:
import datetime as dt

# Get today's date as a Pandas datetime object
first_day = '7/14/2021'
days = (pd.to_datetime(dt.datetime.today().strftime('%Y-%m-%d')) - pd.to_datetime(first_day)).days
days

799

In [27]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [28]:
coin_list = cg.get_coins_list()

coins = pd.DataFrame(coin_list)

coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0vix-protocol,vix,0VIX Protocol
4,0x,zrx,0x Protocol
...,...,...,...
10089,zynecoin,zyn,Zynecoin
10090,zynergy,zyn,Zynergy
10091,zyrri,zyr,Zyrri
10092,zyx,zyx,ZYX


In [29]:
tokens = [
    'GTC',
    'GIV',
    'ENS',
    'KLIMA',
    'RAD',
    'BRIGHT',
    'HONEY',
    'TEC',
    'SEED',
    'PAN',
]

In [30]:
coins = coins[coins['symbol'].str.contains('|'.join(tokens).lower())]
coins

,id,symbol,name
549,arbpanda-ai,aipanda,ArbPanda AI
673,astra-dao,astradao,Astra DAO
749,ausd-seed-acala,aseed,aUSD SEED (Acala)
750,ausd-seed-karura,aseed,aUSD SEED (Karura)
1086,big-panda,panda,Big Panda
...,...,...,...
8783,token-engineering-commons,tec,Token Engineering Commons
8847,toxicgarden-finance-seed,seed,ToxicGarden.finance SEED
8865,tradex-ai,tradex,TradeX AI
8953,tupan,tupan,Tupan


In [31]:
token_ids = [
    'bright-token',
    # 'ethereum-name-service',
    'gitcoin',
    'giveth',
    'rad',
    'honey-finance',
    'token-engineering-commons',
    'metagame',
    
]

exact_match_regex = ['^'+id+'$' for id in token_ids]

In [32]:
coins[coins['id'].str.match('|'.join(exact_match_regex).lower())]

,id,symbol,name
1470,bright-token,bright,BrightID
3706,gitcoin,gtc,Gitcoin
3710,giveth,giv,Giveth
4150,honey-finance,honey,Honey Finance
5498,metagame,seed,MetaGame
7205,rad,rad,RAD
8783,token-engineering-commons,tec,Token Engineering Commons


In [33]:
token_id = 'bright-token'
quote = 'ETH'
days = 730

# Pull prices from coin gecko
prices = cg.get_coin_market_chart_by_id(token_id, quote, days)

# Little bit of clean up
prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
prices = {a:[i[1] for i in b] for a,b in prices.items()}

# Make a dataframe time series
prices_df = pd.DataFrame(prices)

prices_df['token_id'] = token_id

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000268,0.0,50.559488,1632268800000,bright-token
1,0.000254,0.0,46.795054,1632355200000,bright-token
2,0.000233,0.0,41.993178,1632441600000,bright-token
3,0.000223,0.0,52.737524,1632528000000,bright-token
4,0.000218,0.0,32.990833,1632614400000,bright-token
...,...,...,...,...,...
726,0.000022,0.0,0.095688,1694995200000,bright-token
727,0.000021,0.0,0.129893,1695081600000,bright-token
728,0.000021,0.0,0.422886,1695168000000,bright-token
729,0.000022,0.0,5.075154,1695254400000,bright-token


In [34]:
def get_price_history(token_id: str, quote: str, days: int) -> pd.DataFrame:
    # Pull prices from coin gecko
    prices = cg.get_coin_market_chart_by_id(token_id, quote, days)
    
    # Little bit of clean up
    prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
    prices = {a:[i[1] for i in b] for a,b in prices.items()}
    
    # Make a dataframe time series
    prices_df = pd.DataFrame(prices)
    
    prices_df['token_id'] = token_id
    
    return prices_df

In [35]:
token_ids

['bright-token',
 'gitcoin',
 'giveth',
 'rad',
 'honey-finance',
 'token-engineering-commons',
 'metagame']

In [36]:
df = get_price_history(token_ids[1], quote='ETH', days=days)
df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.002481,35306.942321,4790.914111,1632268800000,gitcoin
1,0.002575,36720.182707,3522.080068,1632355200000,gitcoin
2,0.002610,37094.495294,3388.123392,1632441600000,gitcoin
3,0.002400,34072.894891,6061.082234,1632528000000,gitcoin
4,0.002345,33319.882879,4225.775961,1632614400000,gitcoin
...,...,...,...,...,...
726,0.000545,33133.303592,4506.285936,1694995200000,gitcoin
727,0.000545,33213.900903,2625.051774,1695081600000,gitcoin
728,0.000546,33292.533876,3384.170627,1695168000000,gitcoin
729,0.000564,34312.601670,4086.916992,1695254400000,gitcoin


In [77]:
tec_data = pd.read_csv('data/TEC-prices.csv')

tec_data['timestamp'] = pd.to_datetime(tec_data['Date (00:00 UTC,Coingecko)']).dt.normalize()

tec_data['prices'] = tec_data['TEC price (ETH)']
tec_data['token_id'] = 'token-engineering-commons'
tec_data['market_caps'] = 0
tec_data['total_volumes'] = 0

tec_data = tec_data[prices_df[prices_df['token_id']=='token-engineering-commons'].columns]

tec_data

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000514,0,0,2021-07-14,token-engineering-commons
1,0.000501,0,0,2021-07-15,token-engineering-commons
2,0.000523,0,0,2021-07-16,token-engineering-commons
3,0.000534,0,0,2021-07-17,token-engineering-commons
4,0.000526,0,0,2021-07-18,token-engineering-commons
...,...,...,...,...,...
724,0.000269,0,0,2023-07-08,token-engineering-commons
725,0.000266,0,0,2023-07-09,token-engineering-commons
726,0.000271,0,0,2023-07-10,token-engineering-commons
727,0.000265,0,0,2023-07-11,token-engineering-commons


In [98]:
data = []
for token_id in token_ids:
    print(f"Fetching data for token_id: {token_id}")
    df = get_price_history(token_id, quote='ETH', days=days)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.normalize()

    if token_id == 'token-engineering-commons':
        df = df.set_index('timestamp').combine_first(tec_data.set_index('timestamp')).reset_index()
    data.append(df)

print(f"Fetching data for ETH/USD")
df = get_price_history('ethereum', quote='usd', days=days)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.normalize()
data.append(df)

Fetching data for token_id: bright-token
Fetching data for token_id: gitcoin
Fetching data for token_id: giveth
Fetching data for token_id: rad
Fetching data for token_id: honey-finance
Fetching data for token_id: token-engineering-commons
Fetching data for token_id: metagame
Fetching data for ETH/USD


In [99]:
# Concatenate all of the data into one dataframe
prices_df = pd.concat(data)
prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000268,0.000000e+00,5.055949e+01,2021-09-22,bright-token
1,0.000254,0.000000e+00,4.679505e+01,2021-09-23,bright-token
2,0.000233,0.000000e+00,4.199318e+01,2021-09-24,bright-token
3,0.000223,0.000000e+00,5.273752e+01,2021-09-25,bright-token
4,0.000218,0.000000e+00,3.299083e+01,2021-09-26,bright-token
...,...,...,...,...,...
726,1622.478777,1.948948e+11,4.510503e+09,2023-09-18,ethereum
727,1636.634609,1.968665e+11,9.941459e+09,2023-09-19,ethereum
728,1643.698418,1.975794e+11,7.059055e+09,2023-09-20,ethereum
729,1621.532955,1.952383e+11,8.720556e+09,2023-09-21,ethereum


In [100]:
prices_df.dtypes

prices                  float64
market_caps             float64
total_volumes           float64
timestamp        datetime64[ns]
token_id                 object
dtype: object

In [101]:
prices_df[prices_df['token_id']=='token-engineering-commons'].hvplot.step(x='timestamp', y='prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

In [102]:
import hvplot.pandas

In [103]:
prices_df.hvplot.step(x='timestamp', y='prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

Create normalized prices.

In [104]:
prices_df['normalized_prices'] = prices_df['prices'] / prices_df.groupby('token_id')['prices'].transform('first')

In [105]:
prices_df.hvplot.step(x='timestamp', y='normalized_prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (normalized_prices)

Giveth has some anomolous behavior near the beginning. I'm going to remove the first 20 days of giveth.

In [106]:
import holoviews as hv

prices_df_adjusted = prices_df.copy(deep=True)

prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'] = prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'].iloc[20:]

prices_df_adjusted['normalized_prices'] = prices_df_adjusted['prices'] / prices_df_adjusted.groupby('token_id')['prices'].transform('first')

In [107]:
import datetime as dt

In [108]:
d = dt.date.today()

In [109]:
import datetime

def ordinal(n):
    """Return number with ordinal suffix."""
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

def formatted_date(past=False):
    """Return today's date in the desired format."""
    today = datetime.date.today()
    month = today.strftime('%b')
    day = ordinal(today.day)
    year = today.year
    if past:
        year = year - 2
    return f"{month} {day} {year}"

print(formatted_date())

Sep 21st 2023


In [110]:
today = formatted_date()
past = formatted_date(past=True)

In [155]:
plot = prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].hvplot.line(logy=False, legend='top_right', grid=True, rot=45, title=f'Public Goods Tokens Normalized against Ethereum {past} - {today}',x='timestamp', y='normalized_prices', by='token_id', width=1400, height=600, line_width=3) * hv.HLine(1)
plot

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [156]:
plot.opts(xticks=[(date, date.strftime('%B %Y')) for date in pd.date_range(start=prices_df_adjusted['timestamp'].min(), end=prices_df_adjusted['timestamp'].max(), freq='MS')])

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [113]:
prices_df_adjusted['returns'] = prices_df_adjusted.groupby('token_id')['normalized_prices'].transform(lambda x: x.bfill().pct_change()+1)

In [114]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000268,0.000000e+00,5.055949e+01,2021-09-22,bright-token,1.000000,NaN
1,0.000254,0.000000e+00,4.679505e+01,2021-09-23,bright-token,0.947669,0.947669
2,0.000233,0.000000e+00,4.199318e+01,2021-09-24,bright-token,0.870243,0.918298
3,0.000223,0.000000e+00,5.273752e+01,2021-09-25,bright-token,0.830343,0.954152
4,0.000218,0.000000e+00,3.299083e+01,2021-09-26,bright-token,0.814709,0.981171
...,...,...,...,...,...,...,...
726,1622.478777,1.948948e+11,4.510503e+09,2023-09-18,ethereum,0.591258,0.992694
727,1636.634609,1.968665e+11,9.941459e+09,2023-09-19,ethereum,0.596417,1.008725
728,1643.698418,1.975794e+11,7.059055e+09,2023-09-20,ethereum,0.598991,1.004316
729,1621.532955,1.952383e+11,8.720556e+09,2023-09-21,ethereum,0.590914,0.986515


In [115]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
token-engineering-commons,1.002275,0.118033,0.019273
gitcoin,1.000180,0.079937,0.002250
giveth,0.999486,0.166895,-0.003077
bright-token,0.999211,0.076523,-0.010309
honey-finance,0.997760,0.052231,-0.042885
metagame,0.996508,0.061859,-0.056451
rad,0.993864,0.087501,-0.070120


In [116]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').apply(lambda x: x['normalized_prices'].iloc[-1] / x['returns'].std()).sort_values(ascending=False)

token_id
token-engineering-commons    4.250818
honey-finance                2.801268
gitcoin                      2.792936
metagame                     1.743220
bright-token                 1.075661
giveth                       0.133171
rad                          0.049106
dtype: float64

In [117]:
prices_df_adjusted.groupby('token_id')['normalized_prices'].apply(lambda x: (x.iloc[-1] / x.std())).sort_values(ascending=False)

token_id
ethereum                     1.680452
token-engineering-commons    0.855577
gitcoin                      0.684005
bright-token                 0.489994
metagame                     0.426117
honey-finance                0.322043
giveth                       0.093411
rad                          0.017528
Name: normalized_prices, dtype: float64

In [118]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000268,0.000000e+00,5.055949e+01,2021-09-22,bright-token,1.000000,NaN
1,0.000254,0.000000e+00,4.679505e+01,2021-09-23,bright-token,0.947669,0.947669
2,0.000233,0.000000e+00,4.199318e+01,2021-09-24,bright-token,0.870243,0.918298
3,0.000223,0.000000e+00,5.273752e+01,2021-09-25,bright-token,0.830343,0.954152
4,0.000218,0.000000e+00,3.299083e+01,2021-09-26,bright-token,0.814709,0.981171
...,...,...,...,...,...,...,...
726,1622.478777,1.948948e+11,4.510503e+09,2023-09-18,ethereum,0.591258,0.992694
727,1636.634609,1.968665e+11,9.941459e+09,2023-09-19,ethereum,0.596417,1.008725
728,1643.698418,1.975794e+11,7.059055e+09,2023-09-20,ethereum,0.598991,1.004316
729,1621.532955,1.952383e+11,8.720556e+09,2023-09-21,ethereum,0.590914,0.986515


In [119]:
prices_df_adjusted.reset_index()['returns'].bfill().idxmax()

1628

In [120]:
prices_df_adjusted.iloc[4484]

prices                       4637.121617
market_caps          549758919636.061768
total_volumes         31531135168.773865
timestamp            2021-12-01 00:00:00
token_id                        ethereum
normalized_prices               1.689845
returns                         1.043333
Name: 70, dtype: object

In [121]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000268,0.000000e+00,5.055949e+01,2021-09-22,bright-token,1.000000,NaN
1,0.000254,0.000000e+00,4.679505e+01,2021-09-23,bright-token,0.947669,0.947669
2,0.000233,0.000000e+00,4.199318e+01,2021-09-24,bright-token,0.870243,0.918298
3,0.000223,0.000000e+00,5.273752e+01,2021-09-25,bright-token,0.830343,0.954152
4,0.000218,0.000000e+00,3.299083e+01,2021-09-26,bright-token,0.814709,0.981171
...,...,...,...,...,...,...,...
726,1622.478777,1.948948e+11,4.510503e+09,2023-09-18,ethereum,0.591258,0.992694
727,1636.634609,1.968665e+11,9.941459e+09,2023-09-19,ethereum,0.596417,1.008725
728,1643.698418,1.975794e+11,7.059055e+09,2023-09-20,ethereum,0.598991,1.004316
729,1621.532955,1.952383e+11,8.720556e+09,2023-09-21,ethereum,0.590914,0.986515


In [122]:
prices_df_adjusted[prices_df_adjusted['token_id']=='token-engineering-commons']

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000514,0.0,0.000000,2021-07-14,token-engineering-commons,1.000000,NaN
1,0.000501,0.0,0.000000,2021-07-15,token-engineering-commons,0.973323,0.973323
2,0.000523,0.0,0.000000,2021-07-16,token-engineering-commons,1.017616,1.045507
3,0.000534,0.0,0.000000,2021-07-17,token-engineering-commons,1.037449,1.019489
4,0.000526,0.0,0.000000,2021-07-18,token-engineering-commons,1.023449,0.986506
...,...,...,...,...,...,...,...
794,0.000246,0.0,0.095854,2023-09-17,token-engineering-commons,0.479091,0.980016
795,0.000263,0.0,0.008110,2023-09-18,token-engineering-commons,0.511465,1.067573
796,0.000243,0.0,0.001919,2023-09-19,token-engineering-commons,0.473313,0.925406
797,0.000247,0.0,0.007259,2023-09-21,token-engineering-commons,0.480320,1.014804


In [123]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last').sort_values(ascending=False).to_frame(name=f'Return {past} - {today}').reset_index()

,token_id,Return Sep 21st 2021 - Sep 21st 2023
0,token-engineering-commons,0.501739
1,gitcoin,0.223258
2,honey-finance,0.146314
3,metagame,0.107835
4,bright-token,0.082312
5,giveth,0.022226
6,rad,0.004297


In [124]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std').sort_values(ascending=True).to_frame(name=f'Volatility {past} - {today}').reset_index()

,token_id,Volatility Sep 21st 2021 - Sep 21st 2023
0,honey-finance,0.052231
1,metagame,0.061859
2,bright-token,0.076523
3,gitcoin,0.079937
4,rad,0.087501
5,token-engineering-commons,0.118033
6,giveth,0.166895


In [125]:
(prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last') / prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std')).sort_values(ascending=False).to_frame(name=f'Pseudo Sharpe {past} - {today}').reset_index()

,token_id,Pseudo Sharpe Sep 21st 2021 - Sep 21st 2023
0,token-engineering-commons,4.250818
1,honey-finance,2.801268
2,gitcoin,2.792936
3,metagame,1.743220
4,bright-token,1.075661
5,giveth,0.133171
6,rad,0.049106


In [126]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
token-engineering-commons,1.002275,0.118033,0.019273
gitcoin,1.000180,0.079937,0.002250
giveth,0.999486,0.166895,-0.003077
bright-token,0.999211,0.076523,-0.010309
honey-finance,0.997760,0.052231,-0.042885
metagame,0.996508,0.061859,-0.056451
rad,0.993864,0.087501,-0.070120


In [127]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000268,0.000000e+00,5.055949e+01,2021-09-22,bright-token,1.000000,NaN
1,0.000254,0.000000e+00,4.679505e+01,2021-09-23,bright-token,0.947669,0.947669
2,0.000233,0.000000e+00,4.199318e+01,2021-09-24,bright-token,0.870243,0.918298
3,0.000223,0.000000e+00,5.273752e+01,2021-09-25,bright-token,0.830343,0.954152
4,0.000218,0.000000e+00,3.299083e+01,2021-09-26,bright-token,0.814709,0.981171
...,...,...,...,...,...,...,...
726,1622.478777,1.948948e+11,4.510503e+09,2023-09-18,ethereum,0.591258,0.992694
727,1636.634609,1.968665e+11,9.941459e+09,2023-09-19,ethereum,0.596417,1.008725
728,1643.698418,1.975794e+11,7.059055e+09,2023-09-20,ethereum,0.598991,1.004316
729,1621.532955,1.952383e+11,8.720556e+09,2023-09-21,ethereum,0.590914,0.986515


In [128]:
prices_df_adjusted['returns'].rolling(7).std()

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
726    0.022188
727    0.011924
728    0.007392
729    0.009711
730    0.011609
Name: returns, Length: 5145, dtype: float64

In [129]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000268,0.000000e+00,5.055949e+01,2021-09-22,bright-token,1.000000,NaN
1,0.000254,0.000000e+00,4.679505e+01,2021-09-23,bright-token,0.947669,0.947669
2,0.000233,0.000000e+00,4.199318e+01,2021-09-24,bright-token,0.870243,0.918298
3,0.000223,0.000000e+00,5.273752e+01,2021-09-25,bright-token,0.830343,0.954152
4,0.000218,0.000000e+00,3.299083e+01,2021-09-26,bright-token,0.814709,0.981171
...,...,...,...,...,...,...,...
726,1622.478777,1.948948e+11,4.510503e+09,2023-09-18,ethereum,0.591258,0.992694
727,1636.634609,1.968665e+11,9.941459e+09,2023-09-19,ethereum,0.596417,1.008725
728,1643.698418,1.975794e+11,7.059055e+09,2023-09-20,ethereum,0.598991,1.004316
729,1621.532955,1.952383e+11,8.720556e+09,2023-09-21,ethereum,0.590914,0.986515


In [130]:
prices_df_adjusted_tokens = prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].copy(deep=True)

In [131]:
prices_df_adjusted_tokens['volatility'] = prices_df_adjusted_tokens.groupby('token_id')['returns'].transform(
    lambda x: x.rolling(90).std().bfill()
)

In [132]:
prices_df_adjusted_tokens

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns,volatility
0,0.000268,0.0,50.559488,2021-09-22,bright-token,1.000000,NaN,0.180199
1,0.000254,0.0,46.795054,2021-09-23,bright-token,0.947669,0.947669,0.180199
2,0.000233,0.0,41.993178,2021-09-24,bright-token,0.870243,0.918298,0.180199
3,0.000223,0.0,52.737524,2021-09-25,bright-token,0.830343,0.954152,0.180199
4,0.000218,0.0,32.990833,2021-09-26,bright-token,0.814709,0.981171,0.180199
...,...,...,...,...,...,...,...,...
402,0.000954,0.0,0.009536,2023-09-15,metagame,0.108386,0.999903,0.057138
403,0.000947,0.0,0.094714,2023-09-17,metagame,0.107656,0.993259,0.057136
404,0.000947,0.0,0.094714,2023-09-18,metagame,0.107656,1.000000,0.056824
405,0.000949,0.0,0.094871,2023-09-18,metagame,0.107835,1.001661,0.056825


In [133]:
prices_df_adjusted_tokens.hvplot.line(
    logy=True, 
    legend='top_right', 
    grid=True, rot=45, 
    title=f'Public Goods Tokens 90 Day Volatility Against Ethereum {past} - {today}',
    x='timestamp', y='volatility', by='token_id', width=1400, height=600, line_width=3) 

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (volatility)

In [134]:
prices_df_adjusted_tokens

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns,volatility
0,0.000268,0.0,50.559488,2021-09-22,bright-token,1.000000,NaN,0.180199
1,0.000254,0.0,46.795054,2021-09-23,bright-token,0.947669,0.947669,0.180199
2,0.000233,0.0,41.993178,2021-09-24,bright-token,0.870243,0.918298,0.180199
3,0.000223,0.0,52.737524,2021-09-25,bright-token,0.830343,0.954152,0.180199
4,0.000218,0.0,32.990833,2021-09-26,bright-token,0.814709,0.981171,0.180199
...,...,...,...,...,...,...,...,...
402,0.000954,0.0,0.009536,2023-09-15,metagame,0.108386,0.999903,0.057138
403,0.000947,0.0,0.094714,2023-09-17,metagame,0.107656,0.993259,0.057136
404,0.000947,0.0,0.094714,2023-09-18,metagame,0.107656,1.000000,0.056824
405,0.000949,0.0,0.094871,2023-09-18,metagame,0.107835,1.001661,0.056825


In [135]:
df = prices_df_adjusted_tokens.pivot_table(index='timestamp', columns='token_id', values='returns').resample('D').mean()
df

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
timestamp,,,,,,,
2021-07-15,NaN,NaN,NaN,NaN,NaN,NaN,0.973323
2021-07-16,NaN,NaN,NaN,NaN,NaN,NaN,1.045507
2021-07-17,NaN,NaN,NaN,NaN,NaN,NaN,1.019489
2021-07-18,NaN,NaN,NaN,NaN,NaN,NaN,0.986506
2021-07-19,NaN,NaN,NaN,NaN,NaN,NaN,0.996922
...,...,...,...,...,...,...,...
2023-09-17,1.011821,1.013240,1.012888,1.001825,0.993259,NaN,0.980016
2023-09-18,0.991903,0.959145,0.984164,1.025487,1.000830,NaN,1.067573
2023-09-19,0.982395,0.999016,0.991319,0.971290,1.000000,NaN,0.925406


In [136]:
df.isna().sum()

token_id
bright-token                  70
gitcoin                       70
giveth                       181
honey-finance                265
metagame                     394
rad                          231
token-engineering-commons      2
dtype: int64

In [137]:
df.cumprod().bfill().hvplot.line(logy=True)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (value)

In [138]:
cov = df.corr()
cov

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
token_id,,,,,,,
bright-token,1.000000,0.000573,0.014910,0.152926,-0.019399,0.089579,0.071946
gitcoin,0.000573,1.000000,-0.034533,-0.100205,0.010460,-0.019779,-0.002253
giveth,0.014910,-0.034533,1.000000,0.085661,0.336771,0.025880,0.001140
honey-finance,0.152926,-0.100205,0.085661,1.000000,0.174311,0.449631,0.274159
metagame,-0.019399,0.010460,0.336771,0.174311,1.000000,0.083734,0.005687
rad,0.089579,-0.019779,0.025880,0.449631,0.083734,1.000000,0.023914
token-engineering-commons,0.071946,-0.002253,0.001140,0.274159,0.005687,0.023914,1.000000


In [139]:
cov.hvplot.heatmap(cmap='reds', colorbar=True, rot=45, width=800, height=500, title="Public Goods Tokens Correlation")

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)


:HeatMap   [columns,index]   (value)

In [140]:
df.mean()

token_id
bright-token                 0.999192
gitcoin                      1.000170
giveth                       0.999495
honey-finance                0.997735
metagame                     0.996497
rad                          0.993829
token-engineering-commons    1.002240
dtype: float64

# This section needs to be corrected.

Don't impute. Don't dropna. 

Start with prices, backfill missing data. Then compute returns.

In [141]:
import numpy as np

In [142]:
df_filled = df.where(~df.isna(), np.random.normal(df.mean(), df.std(), size=df.shape))

In [143]:
df_filled = df.dropna()

In [144]:
df_filled.cumprod().hvplot.line(logy=True)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (value)

In [145]:
df_filled

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
timestamp,,,,,,,
2022-04-09,0.949924,0.927148,1.002523,1.041729,1.145661,0.987208,1.016271
2022-04-10,0.968138,1.038673,0.976691,0.928047,1.000000,0.783287,0.958001
2022-04-11,0.984979,0.976918,1.060522,1.225002,0.998845,0.910245,1.019078
2022-04-12,1.110989,0.943426,0.985020,1.091937,1.110623,1.009557,1.010602
2022-04-13,0.889215,1.046814,0.937338,0.955503,0.999818,0.936462,0.984746
...,...,...,...,...,...,...,...
2023-08-27,1.004554,1.006104,1.001380,0.999220,1.037751,0.999940,1.007593
2023-08-30,0.980725,1.011458,1.001554,0.940267,0.926243,1.000000,1.005719
2023-08-31,1.021863,1.009520,0.995744,1.019316,1.000000,1.012279,0.987711


In [146]:
N = 1000
portfolios = pd.DataFrame(np.random.rand(N, df_filled.shape[1]), columns=df_filled.columns)
portfolios = (portfolios.T / portfolios.sum(axis=1)).T
portfolios

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
0,0.000853,0.074251,0.228369,0.202490,0.262612,0.227130,0.004295
1,0.148547,0.107981,0.256571,0.075164,0.071598,0.200619,0.139520
2,0.260671,0.023711,0.190869,0.146134,0.081862,0.119985,0.176769
3,0.178089,0.212690,0.069950,0.180006,0.068158,0.205384,0.085724
4,0.084493,0.170560,0.086873,0.215077,0.174365,0.136556,0.132075
...,...,...,...,...,...,...,...
995,0.041854,0.214011,0.043016,0.042805,0.181588,0.312401,0.164324
996,0.184029,0.118263,0.203724,0.041839,0.181055,0.172854,0.098236
997,0.094040,0.253059,0.219904,0.235008,0.048714,0.045517,0.103759
998,0.168946,0.146111,0.042225,0.232622,0.114847,0.231755,0.063493


In [147]:
outcomes = df_filled @ portfolios.T
outcomes

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-04-09,1.038990,0.998611,1.005064,0.991971,1.018338,0.996141,1.023660,1.000349,1.010046,0.991135,...,1.027044,1.013156,1.020933,1.010729,1.018139,1.009335,1.010188,0.995418,1.005506,1.044208
2022-04-10,0.933549,0.938718,0.944222,0.939859,0.951263,0.947375,0.953538,0.939362,0.902033,0.921680,...,0.950672,0.956271,0.966143,0.961507,0.932856,0.928257,0.949366,0.970533,0.929655,0.943534
2022-04-11,1.037048,1.012289,1.032478,1.020274,1.038506,0.994657,1.032105,1.024594,1.038888,1.010937,...,1.055649,1.012974,1.040852,1.004164,1.000480,0.981552,1.002400,1.056771,1.029263,1.020108
2022-04-12,1.042357,1.024762,1.050243,1.033646,1.040195,1.030996,1.078789,1.033609,1.062126,1.038219,...,1.064672,1.030350,1.059952,1.033029,1.034671,1.020645,1.037252,1.021357,1.046832,1.042337
2022-04-13,0.965516,0.954288,0.943434,0.963465,0.972887,0.961912,0.946523,0.955993,0.941448,0.947538,...,0.961486,0.973074,0.952410,0.974008,0.955081,0.978393,0.958007,0.972708,0.959412,0.969055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27,1.010547,1.005381,1.005907,1.005277,1.008955,1.007947,1.010984,1.005597,1.004341,1.003962,...,1.009873,1.009742,1.009715,1.010633,1.009288,1.009607,1.009379,1.004717,1.006342,1.013042
2023-08-30,0.969749,0.989800,0.981788,0.983824,0.975508,0.986017,0.967644,0.983059,0.972475,0.986300,...,0.963678,0.983158,0.975920,0.983193,0.981393,0.986702,0.982833,0.984391,0.976481,0.972723
2023-08-31,1.006401,1.005384,1.007236,1.010566,1.007309,1.008343,1.009840,1.008813,1.013534,1.010206,...,1.011715,1.004341,1.004858,1.004832,1.005770,1.005413,1.006006,1.007352,1.011464,1.002652


In [148]:
means = outcomes.mean()
stds = outcomes.std()
ratios = means / stds

In [149]:
markowitz = pd.DataFrame({'mean':means, 'std':stds, 'sharpe':ratios})

In [150]:
markowitz.hvplot.scatter(y='mean',x='std',c='sharpe', cmap='RdYlGn', colorbar=True, size=10, width=500, height=500)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Scatter   [std]   (mean,sharpe)

In [151]:
ratios.argmax()

409

In [152]:
portfolios.iloc[923]

token_id
bright-token                 0.266245
gitcoin                      0.097314
giveth                       0.016006
honey-finance                0.154774
metagame                     0.200823
rad                          0.102568
token-engineering-commons    0.162269
Name: 923, dtype: float64